In [1]:
# Cell 1: install dependencies
!pip install -q gradio pandas python-dateutil langchain openai pytest


In [2]:
# Cell 2: imports, dataset creation, admin accounts
import os
import json
import csv
import time
from datetime import datetime, timedelta
from getpass import getpass

import pandas as pd

# Create dataset (same sample used earlier)
data = [
    (1, "Asha Kumar", 8, "A", False, "2025-11-05", "Math Quiz 1", "2025-11-20", 78, "2025-11-06"),
    (2, "Ravi Patel", 8, "A", True,  "2025-11-10", "Science Quiz 1", "2025-11-19", 85, "2025-11-08"),
    (3, "Sneha Reddy", 8, "B", False, "2025-11-01", "Math Quiz 1", "2025-11-20", 65, "2025-11-05"),
    (4, "P. Venkatesh", 7, "A", True, "2025-11-09", "History Quiz",   "2025-11-24", 92, "2025-11-07"),
    (5, "Meera Iyer", 7, "B", False, "2025-10-30", "Geography Quiz", "2025-11-26", 74, "2025-10-29"),
    (6, "Karan Singh", 9, "A", True, "2025-11-11", "Math Quiz 1",    "2025-11-20", 88, "2025-11-10"),
    (7, "Latha Nair", 9, "B", True, "2025-11-12", "Science Quiz 1", "2025-11-19", 90, "2025-11-09"),
    (8, "Santhosh G", 8, "B", True, "2025-11-07", "History Quiz",   "2025-11-24", 81, "2025-11-02"),
    (9, "Divya Sharma",8, "A", False, "2025-11-03", "Geography Quiz", "2025-11-26", 70, "2025-11-01"),
    (10, "Ajay Kumar",7, "A", True, "2025-11-12", "Science Quiz 1", "2025-11-19", 76, "2025-11-11"),
    (11, "Priya Das", 8, "B", False, "2025-11-04", "Math Quiz 1",    "2025-11-20", 69, "2025-11-06"),
    (12, "Rohan Mehta",8, "A", True, "2025-11-10", "History Quiz",   "2025-11-24", 83, "2025-11-08"),
]

df = pd.DataFrame(data, columns=[
    "student_id","name","grade","class","homework_submitted","last_submission_date",
    "upcoming_quiz_name","upcoming_quiz_date","last_quiz_score","last_quiz_date"
])
df['last_submission_date'] = pd.to_datetime(df['last_submission_date'])
df['upcoming_quiz_date'] = pd.to_datetime(df['upcoming_quiz_date'])
df['last_quiz_date'] = pd.to_datetime(df['last_quiz_date'])

csv_path = "/content/dumroo_students.csv"
df.to_csv(csv_path, index=False)
print("Dataset created:", csv_path)
display(df.head(12))

# Admin accounts (simulated)
# key = admin username; value includes scope grade/class
admins = {
    "admin_grade8": {"grade": 8, "class": None, "display": "Grade 8 Admin"},
    "admin_7A": {"grade": 7, "class": "A", "display": "Grade 7A Admin"},
    "admin_region": {"grade": None, "class": None, "display": "Regional Supervisor"},
}

print("Available admins:")
for k,v in admins.items():
    print(f" - {k}: {v['display']} (grade={v['grade']}, class={v['class']})")


Dataset created: /content/dumroo_students.csv


,student_id,name,grade,class,homework_submitted,last_submission_date,upcoming_quiz_name,upcoming_quiz_date,last_quiz_score,last_quiz_date
0,1,Asha Kumar,8,A,False,2025-11-05,Math Quiz 1,2025-11-20,78,2025-11-06
1,2,Ravi Patel,8,A,True,2025-11-10,Science Quiz 1,2025-11-19,85,2025-11-08
2,3,Sneha Reddy,8,B,False,2025-11-01,Math Quiz 1,2025-11-20,65,2025-11-05
3,4,P. Venkatesh,7,A,True,2025-11-09,History Quiz,2025-11-24,92,2025-11-07
4,5,Meera Iyer,7,B,False,2025-10-30,Geography Quiz,2025-11-26,74,2025-10-29
5,6,Karan Singh,9,A,True,2025-11-11,Math Quiz 1,2025-11-20,88,2025-11-10
6,7,Latha Nair,9,B,True,2025-11-12,Science Quiz 1,2025-11-19,90,2025-11-09
7,8,Santhosh G,8,B,True,2025-11-07,History Quiz,2025-11-24,81,2025-11-02
8,9,Divya Sharma,8,A,False,2025-11-03,Geography Quiz,2025-11-26,70,2025-11-01
9,10,Ajay Kumar,7,A,True,2025-11-12,Science Quiz 1,2025-11-19,76,2025-11-11


Available admins:
 - admin_grade8: Grade 8 Admin (grade=8, class=None)
 - admin_7A: Grade 7A Admin (grade=7, class=A)
 - admin_region: Regional Supervisor (grade=None, class=None)


In [3]:
# Cell 3: utilities

AUDIT_LOG = "/content/audit_queries.csv"

def scope_filter(df_in, admin_scope):
    out = df_in.copy()
    if admin_scope.get("grade") is not None:
        out = out[out["grade"] == admin_scope["grade"]]
    if admin_scope.get("class") is not None:
        out = out[out["class"] == admin_scope["class"]]
    return out

def parse_time_window(q):
    """
    Resolves 'next week' and 'last week' relative to a fixed ref date (for reproducible demo).
    Returns (start_date, end_date) as date objects, or (None, None).
    """
    ref = datetime(2025, 11, 13)
    q = q.lower()
    if "next week" in q:
        days_to_next_monday = (7 - ref.weekday()) % 7
        if days_to_next_monday == 0:
            days_to_next_monday = 7
        start = (ref + timedelta(days=days_to_next_monday)).date()
        end = start + timedelta(days=6)
        return start, end
    if "last week" in q:
        last_monday = (ref - timedelta(days=ref.weekday()+7)).date()
        last_sunday = last_monday + timedelta(days=6)
        return last_monday, last_sunday
    return None, None

def audit_log(admin_username, question, parsed_filter, result_count):
    """
    Append audit row: timestamp, admin_username, question, parsed_filter (json), result_count
    """
    row = [time.strftime("%Y-%m-%d %H:%M:%S"), admin_username, question, json.dumps(parsed_filter), result_count]
    write_header = not os.path.exists(AUDIT_LOG)
    with open(AUDIT_LOG, "a", newline="") as f:
        w = csv.writer(f)
        if write_header:
            w.writerow(["timestamp","admin","question","parsed_filter","result_count"])
        w.writerow(row)


In [4]:
# Cell 4: rule-based parser (uses admin_username)
import re

def answer_rule_based(question, admin_username):
    admin_scope = admins[admin_username]
    scoped = scope_filter(df, admin_scope)
    q = question.lower()

    # 1) missing homework
    if re.search(r"(haven('|’)t submitted|haven't submitted|not submitted|missing homework)", q):
        res = scoped[scoped["homework_submitted"] == False]
        audit_log(admin_username, question, {"intent":"missing_homework"}, len(res))
        return {"type":"table","title":"Students Missing Homework","data":res}

    # 2) performance for grade N from last week (explicit)
    m = re.search(r"performance.*grade\s*(\d+).*last week", q)
    if m:
        grade = int(m.group(1))
        # enforce admin scope server-side
        if admin_scope["grade"] is not None and admin_scope["grade"] != grade:
            return {"type":"error","message":f"Access denied: you are scoped to grade {admin_scope['grade']}."}
        start, end = parse_time_window(q)
        scoped_grade = scoped[scoped["grade"] == grade]
        if start and end:
            mask = (scoped_grade["last_quiz_date"].dt.date >= start) & (scoped_grade["last_quiz_date"].dt.date <= end)
            res = scoped_grade.loc[mask, ["student_id","name","grade","class","last_quiz_score","last_quiz_date"]]
            audit_log(admin_username, question, {"intent":"performance","grade":grade,"time_window":[str(start),str(end)]}, len(res))
            return {"type":"table","title":f"Performance for Grade {grade} ({start} to {end})","data":res}
        else:
            res = scoped_grade[["student_id","name","grade","class","last_quiz_score","last_quiz_date"]]
            audit_log(admin_username, question, {"intent":"performance","grade":grade}, len(res))
            return {"type":"table","title":f"Performance for Grade {grade}","data":res}

    # 3) upcoming quizzes next week
    if re.search(r"(upcoming quizzes|quizzes scheduled).*next week", q):
        start, end = parse_time_window(q)
        if start and end:
            mask = (scoped["upcoming_quiz_date"].dt.date >= start) & (scoped["upcoming_quiz_date"].dt.date <= end)
            res = scoped.loc[mask, ["student_id","name","grade","class","upcoming_quiz_name","upcoming_quiz_date"]].drop_duplicates()
            audit_log(admin_username, question, {"intent":"upcoming_quizzes","time_window":[str(start),str(end)]}, len(res))
            return {"type":"table","title":f"Upcoming quizzes ({start} to {end})","data":res}
        else:
            return {"type":"error","message":"Could not determine the date window for 'next week'."}

    # fallback: show students in scope
    res = scoped[["student_id","name","grade","class","homework_submitted"]].reset_index(drop=True)
    audit_log(admin_username, question, {"intent":"fallback_list"}, len(res))
    return {"type":"table","title":"Students in your scope","data":res}


In [6]:
# Cell 5: LLM parsing utilities and validator (optional, uses OpenAI via LangChain)
!pip install -q langchain langchain-openai

import os
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from datetime import datetime

ALLOWED_INTENTS = {"missing_homework","performance","upcoming_quizzes","list_students","unknown"}
ALLOWED_KEYS = {"intent","grade","class","time_window"}

def validate_llm_json(parsed):
    if not isinstance(parsed, dict):
        raise ValueError("LLM output not a JSON object")
    extra = set(parsed.keys()) - ALLOWED_KEYS
    if extra:
        raise ValueError(f"Unexpected keys in LLM output: {extra}")
    intent = parsed.get("intent") or "unknown"
    if intent not in ALLOWED_INTENTS:
        raise ValueError(f"Invalid intent: {intent}")

    grade = parsed.get("grade")
    if grade is not None:
        if isinstance(grade, str) and grade.isdigit():
            grade = int(grade)
        elif not isinstance(grade, int):
            raise ValueError("grade must be integer or null")

    cls = parsed.get("class")
    if cls is not None and not isinstance(cls, str):
        raise ValueError("class must be string or null")

    tw = parsed.get("time_window")
    if tw is not None:
        if not isinstance(tw, dict):
            raise ValueError("time_window must be object or null")
        if "start" not in tw or "end" not in tw:
            raise ValueError("time_window must include start and end")
        try:
            datetime.fromisoformat(tw["start"])
            datetime.fromisoformat(tw["end"])
        except:
            raise ValueError("Invalid ISO date format in time_window")

    return {"intent":intent, "grade":grade, "class":cls, "time_window":tw}

def enforce_admin_scope(parsed_filters, admin_scope):
    pf = parsed_filters.copy()
    if admin_scope.get("grade") is not None:
        if pf.get("grade") is None:
            pf["grade"] = admin_scope["grade"]
        elif pf["grade"] != admin_scope["grade"]:
            raise PermissionError(f"Access denied: scoped to grade {admin_scope['grade']}")
    if admin_scope.get("class") is not None:
        if pf.get("class") is None:
            pf["class"] = admin_scope["class"]
        elif pf["class"] != admin_scope["class"]:
            raise PermissionError(f"Access denied: scoped to class {admin_scope['class']}")
    return pf

# Lazy creation of chain
_llm_chain = None

def get_langchain_chain():
    global _llm_chain
    if _llm_chain is None:
        if "OPENAI_API_KEY" not in os.environ:
            key = getpass("Enter your OpenAI API key: ")
            os.environ["OPENAI_API_KEY"] = key

        llm = OpenAI(temperature=0)
        prompt = PromptTemplate(
            input_variables=["question","ref_date"],
            template="""
Convert the user's question into STRICT JSON.
Required keys:
- intent: one of missing_homework, performance, upcoming_quizzes, list_students
- grade: integer or null
- class: string or null
- time_window: {"start":"YYYY-MM-DD","end":"YYYY-MM-DD"} or null

Return ONLY JSON.

question: {question}
ref_date: {ref_date}
"""
        )
        _llm_chain = LLMChain(llm=llm, prompt=prompt)
    return _llm_chain

def llm_parse_to_filter(question, ref_date_str="2025-11-13"):
    chain = get_langchain_chain()
    raw = chain.run({"question":question, "ref_date":ref_date_str}).strip()

    # Remove accidental code fences
    if raw.startswith("```"):
        raw = raw.replace("```", "").strip()

    # Parse JSON
    try:
        parsed = json.loads(raw)
    except:
        raise ValueError("LLM did not return valid JSON")

    return validate_llm_json(parsed)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 1.5 MB/s eta 0:00:00


In [7]:
# Cell 6: apply parsed filters and run query (uses audit_log)
def apply_llm_and_run(question, admin_username, df_local):
    admin_scope = admins[admin_username]
    try:
        parsed = llm_parse_to_filter(question)
    except Exception as e:
        return {"type":"error", "message":f"Could not parse question: {e}"}
    try:
        parsed = enforce_admin_scope(parsed, admin_scope)
    except PermissionError as e:
        return {"type":"error", "message":str(e)}
    # apply parsed filters:
    intent = parsed["intent"]
    grade = parsed.get("grade")
    cls = parsed.get("class")
    tw = parsed.get("time_window")
    scoped = df_local.copy()
    if grade is not None:
        scoped = scoped[scoped["grade"] == int(grade)]
    if cls is not None:
        scoped = scoped[scoped["class"] == cls]

    if intent == "missing_homework":
        res = scoped[scoped["homework_submitted"] == False]
        audit_log(admin_username, question, parsed, len(res))
        return {"type":"table","title":"Missing homework (LLM)","data":res}
    if intent == "performance":
        if tw and tw.get("start"):
            start = datetime.fromisoformat(tw["start"]).date()
            end = datetime.fromisoformat(tw["end"]).date()
            mask = (scoped["last_quiz_date"].dt.date >= start) & (scoped["last_quiz_date"].dt.date <= end)
            res = scoped.loc[mask, ["student_id","name","grade","class","last_quiz_score","last_quiz_date"]]
            audit_log(admin_username, question, parsed, len(res))
            return {"type":"table","title":f"Performance ({start}–{end})","data":res}
        else:
            res = scoped[["student_id","name","grade","class","last_quiz_score","last_quiz_date"]]
            audit_log(admin_username, question, parsed, len(res))
            return {"type":"table","title":"Performance (all)","data":res}
    if intent == "upcoming_quizzes":
        if tw and tw.get("start"):
            start = datetime.fromisoformat(tw["start"]).date()
            end = datetime.fromisoformat(tw["end"]).date()
            mask = (scoped["upcoming_quiz_date"].dt.date >= start) & (scoped["upcoming_quiz_date"].dt.date <= end)
            res = scoped.loc[mask, ["student_id","name","grade","class","upcoming_quiz_name","upcoming_quiz_date"]].drop_duplicates()
            audit_log(admin_username, question, parsed, len(res))
            return {"type":"table","title":f"Quizzes ({start}–{end})","data":res}
        else:
            res = scoped[["student_id","name","grade","class","upcoming_quiz_name","upcoming_quiz_date"]].drop_duplicates()
            audit_log(admin_username, question, parsed, len(res))
            return {"type":"table","title":"Upcoming quizzes","data":res}
    # fallback
    res = scoped[["student_id","name","grade","class","homework_submitted"]]
    audit_log(admin_username, question, parsed, len(res))
    return {"type":"table","title":"Students in scope","data":res}


In [9]:
# Cell 7: Gradio UI (final cell)
import gradio as gr
import pandas as pd

def run_query_ui(admin_username, question, use_llm):
    if admin_username not in admins:
        return "Invalid admin selected.", ""
    if use_llm:
        out = apply_llm_and_run(question, admin_username, df)
    else:
        out = answer_rule_based(question, admin_username)
    if out["type"] == "table":
        # return title and HTML table
        return out["title"], out["data"].to_html(index=False)
    else:
        return out.get("message","Sorry, couldn't answer."), ""

def show_audit_log():
    if not os.path.exists(AUDIT_LOG):
        return "Audit log is empty.", ""
    a = pd.read_csv(AUDIT_LOG)
    return "Audit log", a.to_html(index=False)

with gr.Blocks() as ui:
    gr.Markdown("## 🛡 Dumroo Admin Panel — Natural Language Query Demo")
    gr.Markdown("Select an admin (simulated), ask a question. Toggle LLM parser to test LLM->JSON flow.")
    with gr.Row():
        admin_sel = gr.Dropdown(choices=list(admins.keys()), value="admin_grade8", label="Sign in as (admin username)")
        use_llm = gr.Checkbox(label="Use LLM parser (OpenAI)", value=False)
    question = gr.Textbox(label="Your question", placeholder="e.g. Which students haven't submitted their homework yet?")
    run_btn = gr.Button("Run")
    title_out = gr.Text()
    table_out = gr.HTML()
    audit_btn = gr.Button("Show Audit Log")
    run_btn.click(run_query_ui, inputs=[admin_sel, question, use_llm], outputs=[title_out, table_out])
    audit_btn.click(lambda: show_audit_log(), outputs=[title_out, table_out])

print("Launching Gradio UI (this will be the last executed cell).")
ui.launch(share=True)


Launching Gradio UI (this will be the last executed cell).
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed3e4fe0524449a496.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
